In [1]:
import numpy as np                        
import matplotlib.pyplot as plt          
import pyrealsense2 as rs                 
APIpipe = rs.pipeline()
import cv2
import open3d as o3d
import imshowpair
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import glob
import pylab
import scipy.signal



In [2]:
def normalize(x):
    return (x - x.min())/(x.max() - x.min())

In [3]:
def surface_curvature(X,Y,Z):
	(lr,lb)=X.shape
	print(lr)
	print(lb)
	grad = np.gradient
	spacing_x = 100
	spacing_y = 100
	spacing_z = 100
	# FIRST DERIVATIVE
	Xv,Xu=grad(X, spacing_x)
	Yv,Yu=grad(Y, spacing_y)
	Zv,Zu=grad(Z, spacing_z)
	#Second Derivatives
	Xuv,Xuu=grad(Xu, spacing_x)
	Yuv,Yuu=grad(Yu, spacing_y)
	Zuv,Zuu=grad(Zu, spacing_z)   
	Xvv,Xuv=grad(Xv, spacing_x)
	Yvv,Yuv=grad(Yv, spacing_y)
	Zvv,Zuv=grad(Zv, spacing_z) 
	#2D to 1D conversion 
	#Reshape to 1D vectors
	Xu=np.reshape(Xu,lr*lb)
	Yu=np.reshape(Yu,lr*lb)
	Zu=np.reshape(Zu,lr*lb)
	Xv=np.reshape(Xv,lr*lb)
	Yv=np.reshape(Yv,lr*lb)
	Zv=np.reshape(Zv,lr*lb)
	Xuu=np.reshape(Xuu,lr*lb)
	Yuu=np.reshape(Yuu,lr*lb)
	Zuu=np.reshape(Zuu,lr*lb)
	Xuv=np.reshape(Xuv,lr*lb)
	Yuv=np.reshape(Yuv,lr*lb)
	Zuv=np.reshape(Zuv,lr*lb)
	Xvv=np.reshape(Xvv,lr*lb)
	Yvv=np.reshape(Yvv,lr*lb)
	Zvv=np.reshape(Zvv,lr*lb)
	Xu=np.c_[Xu, Yu, Zu]
	Xv=np.c_[Xv, Yv, Zv]
	Xuu=np.c_[Xuu, Yuu, Zuu]
	Xuv=np.c_[Xuv, Yuv, Zuv]
	Xvv=np.c_[Xvv, Yvv, Zvv]
	#% First fundamental Coeffecients of the surface (E,F,G)
	E=np.einsum('ij,ij->i', Xu, Xu) 
	F=np.einsum('ij,ij->i', Xu, Xv) 
	G=np.einsum('ij,ij->i', Xv, Xv) 
	m=np.cross(Xu,Xv,axisa=1, axisb=1) 
	p=np.sqrt(np.einsum('ij,ij->i', m, m)) 
	n=m/np.c_[p,p,p]
	# n is the normal
	#% Second fundamental Coeffecients of the surface (L,M,N), (e,f,g)
	L= np.einsum('ij,ij->i', Xuu, n) #e
	M= np.einsum('ij,ij->i', Xuv, n) #f
	N= np.einsum('ij,ij->i', Xvv, n) #g
	# Alternative formula for gaussian curvature in wiki 
	# K = det(second fundamental) / det(first fundamental)
	#% Gaussian Curvature
	K=(L*N-M**2)/(E*G-F**2)
	K=np.reshape(K,lr*lb)
	#	print(K.size)
	#wiki trace of (second fundamental)(first fundamental inverse)
	#% Mean Curvature
	H = ((E*N + G*L - 2*F*M)/((E*G - F**2)))/2
	print(H.shape)
	H = np.reshape(H,lr*lb)
	#	print(H.size)
	#% Principle Curvatures
	Pmax = H + np.sqrt(H**2 - K)
	Pmin = H - np.sqrt(H**2 - K)
	#[Pmax, Pmin]
	Principal = [Pmax,Pmin]
	return K, H, n, Principal



In [20]:
import os
normal_dir = 'data/normal/'
mean_curvature_dir = 'data/mean_curvature/'
gaussian_curvature_dir = 'data/gaussian_curvature/'

demonstration_dir = 'demonstration/'
if not os.path.exists(demonstration_dir):
  os.makedirs(demonstration_dir)

if not os.path.exists(normal_dir):
  os.makedirs(normal_dir)
if not os.path.exists(mean_curvature_dir):
  os.makedirs(mean_curvature_dir)
if not os.path.exists(gaussian_curvature_dir):
  os.makedirs(gaussian_curvature_dir)
  
# create directory for images and depth maps
unfilled_depth_dir = 'data/unfilled_depth/'
image_dir = 'data/rgb/'
depth_dir = 'data/depth/'

if not os.path.exists(unfilled_depth_dir):
    os.makedirs(unfilled_depth_dir)
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
if not os.path.exists(depth_dir):
    os.makedirs(depth_dir)

bag_files = '03-21-2024-selectedBagFiles/*.bag'

for bag_file in glob.glob(bag_files):
  try:
    pipeline = rs.pipeline()
    config = rs.config()
    colorizer = rs.colorizer()
    config.enable_device_from_file(bag_file)
    profile = pipeline.start(config)
    
    frameset = pipeline.wait_for_frames()
    color_frame = frameset.get_color_frame()
    depth_frame = frameset.get_depth_frame()
    
    pipeline.stop()
    align = rs.align(rs.stream.color)
    aligned_frameset = align.process(frameset)
    aligned_depth_frame = aligned_frameset.get_depth_frame()
    aligned_depth = np.asarray(aligned_depth_frame.get_data())
    h,w = aligned_depth.shape
    
    pc = rs.pointcloud()
    hole_fill = rs.hole_filling_filter(0)
    decimation_filter = rs.decimation_filter(magnitude = 5.0)
    spatial_filter = rs.spatial_filter(smooth_alpha = 1.0, smooth_delta = 50.0, magnitude = 5.0, hole_fill = 0)
    
    temporal_filter = rs.temporal_filter(smooth_alpha = 1.0, smooth_delta = 100, persistence_control = 1)
    
    processed_depth_frame = decimation_filter.process(aligned_frameset.get_depth_frame())
    processed_depth_frame = spatial_filter.process(processed_depth_frame)
    processed_depth_frame = temporal_filter.process(processed_depth_frame)
    processed_depth_frame = hole_fill.process(processed_depth_frame)
    
    processed_depth = np.asanyarray(colorizer.colorize(processed_depth_frame).get_data())
    
    # get the correspond color frame for the processed depth frame after decimation, spatial, and temporal filtering
    # processed_color_frame = decimation_filter.process(aligned_frameset.get_color_frame())
    # processed_color_frame = spatial_filter.process(processed_color_frame)
    # processed_color_frame = temporal_filter.process(processed_color_frame)
    # processed_color_frame = hole_fill.process(processed_color_frame)
    color = np.asarray(color_frame.get_data())
    color = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    color = cv2.resize(color, (processed_depth.shape[1], processed_depth.shape[0]), interpolation=cv2.INTER_AREA)
    

    
    
    processed_depth = cv2.cvtColor(processed_depth, cv2.COLOR_BGR2GRAY)
    pointsContainer = pc.calculate(processed_depth_frame)
    h,w = processed_depth.shape
    points = np.asarray(pointsContainer.get_vertices())
    points = points.view(np.float32).reshape(points.shape + (-1,))
    x = points[:,0]
    y = points[:,1]
    z = points[:,2]
    x = x.reshape((h,w))
    y = y.reshape((h,w))
    z = z.reshape((h,w))
    
    # crop 30% of the image from the left and 15% from the top and bottom
    crop_left = int(w * 0.30)
    crop_top = int(h * 0.15)
    crop_bottom = int(h * 0.15)
    x = x[crop_top:h-crop_bottom, crop_left:w]
    y = y[crop_top:h-crop_bottom, crop_left:w]
    z = z[crop_top:h-crop_bottom, crop_left:w]

    # crop the corresponding color image and processed depth image
    color = color[crop_top:h-crop_bottom, crop_left:w]
    processed_depth = processed_depth[crop_top:h-crop_bottom, crop_left:w]
        
    h,w = z.shape
    z = cv2.bilateralFilter(z, 6, 11, 11)
    
    K, H, n, Principal = surface_curvature(x,y,z)
    K = K.reshape((h,w))
    H = H.reshape((h,w))
    n = n.reshape((h,w,3))
    
    p_max = Principal[0].reshape((h,w))
    p_min = Principal[1].reshape((h,w))
    
    
    # nan values are present in the normal, mean curvature, and gaussian curvature
    # replace nan values with 0
    n = np.nan_to_num(n)
    H = np.nan_to_num(H)
    K = np.nan_to_num(K)
    
    # normalize the normal, mean curvature, and gaussian curvature
    n = cv2.normalize(n, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    H = cv2.normalize(H, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    K = cv2.normalize(K, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # # convert the normal to an RGB image
    n = cv2.cvtColor(n, cv2.COLOR_BGR2RGB)
    n = n * 255
    n = n.astype(np.uint8)
    
    # # convert the mean curvature and gaussian curvature to a grayscale image
    H = H * 255
    H = H.astype(np.uint8)
    K = K * 255
    K = K.astype(np.uint8)
    # plot img, depth, H, K, n[:,:,2]
    fig = plt.figure()
    # set plt size to 20x20
    fig.set_size_inches(25, 5)
    fig.add_subplot(1, 5, 1)
    plt.imshow(color)
    plt.title('RGB')
    fig.add_subplot(1, 5, 2)
    plt.imshow(processed_depth)
    plt.title('D')
    fig.add_subplot(1, 5, 3)
    plt.imshow(H, cmap='gray')
    plt.title('H')
    fig.add_subplot(1, 5, 4)
    plt.imshow(K, cmap='gray')
    plt.title('K')
    fig.add_subplot(1, 5, 5)
    plt.imshow(n[:,:,2], cmap='gray')
    plt.title('N')
    plt.savefig('demonstration/' + os.path.basename(bag_file[19:-4]) + '_demo.png')
    plt.close()

    # # save the normal, mean curvature, and gaussian curvature as images
    cv2.imwrite(normal_dir + os.path.basename(bag_file[19:-4]) + "_normal.png", n)
    cv2.imwrite(mean_curvature_dir + os.path.basename(bag_file[19:-4]) + "_mean_curvature.png", H)
    cv2.imwrite(gaussian_curvature_dir + os.path.basename(bag_file[19:-4]) + "_gaussian_curvature.png", K)
    # save the color and depth images
    cv2.imwrite(image_dir + os.path.basename(bag_file[19:-4]) + "_color.png", color)
    cv2.imwrite(depth_dir + os.path.basename(bag_file[19:-4]) + "_depth.png", processed_depth)
    cv2.imwrite(unfilled_depth_dir + os.path.basename(bag_file[19:-4]) + "_unfilled_depth.png", aligned_depth)
    
  except Exception as e:
    print(e)
  finally:
    pass


##### UNUSED CODE #####
    # estimate normals
    # pointCloud = o3d.geometry.PointCloud()
    # cropped_points = np.asarray([x.reshape(h*w,),y.reshape(h*w, ),z.reshape(h*w, )]).T
    # pointCloud.points = o3d.utility.Vector3dVector(cropped_points)
    # pointCloud.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = 0.05, max_nn = 10))
    # normals = np.asarray(pointCloud.normals)
    # normals = normals.reshape((h,w,3))
    
    # run poisson reconstruction
    # poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pointCloud, depth = 9, width = 0, scale = 1.1, linear_fit = False)[0]
    # print(poisson_mesh)
    # # o3d.visualization.draw_geometries([poisson_mesh],
    # #                               window_name = 'Poisson Mesh',
    # #                               width = 800,
    # #                               height = 600,
    # #                               left = 50,
    # #                               top = 50)
    # o3d.io.write_triangle_mesh('poisson_mesh.ply', poisson_mesh)
    


102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360,)
102
180
(18360